## Import libraries

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import google.generativeai as genai
from google.colab import userdata
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
import os
import IPython
import time
import re
import shutil
import glob
import numpy as np

## **Gemini configuration**

In [ ]:
# API key
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

genai.configure(api_key = GOOGLE_API_KEY)

In [ ]:
system_instruction = "You are a sustainability expert consultant. You must respond with facts."

In [ ]:
model = genai.GenerativeModel('gemini-1.5-flash-002',system_instruction=system_instruction) # gemini-1.5-flash # gemini-1.5-pro # gemini-pro # gemini-1.5-flash-002

## Adjust model's temperature, top_p, top_k (0 for more deterministic answers)

In [ ]:
generation_config = genai.GenerationConfig(top_p = 0, temperature = 0, top_k= 1)

## Safety settings

In [ ]:
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

## Mount drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Base path
base_path = "/content/drive/My Drive" # Esto no cambiar nunca

# Define the folder path in your Google Drive
general_path = os.path.join(base_path,"TrabajoWater")

# Check if the folder exists, if not, create it
if not os.path.exists(general_path):
    os.makedirs(general_path)
    print(f"Created path {general_path}")

## Response folder

In [ ]:
# Load the Excel
path_excel = os.path.join(general_path, "synergy-parte2.xlsx")
print(f"Loading Excel: {path_excel}")

df = pd.read_excel(path_excel)

print(df.head())


In [ ]:
# Folder where new answers will be saved
interaction_water_folder = os.path.join(general_path,"Raw_Results_Flash_002","Prompt0_synergy2_prueba1")
if not os.path.exists(interaction_water_folder):
    os.makedirs(interaction_water_folder)
    print(f"Created path {interaction_water_folder}")

## Functions

In [ ]:
def map_sdg(sdg):
    sdg = str(sdg).strip().lower()  # Convert to lowercase and remove any extra spaces

    if any(keyword in sdg for keyword in ['no poverty', 'poverty', 'poor']):
        return '1) No Poverty'
    elif any(keyword in sdg for keyword in ['zero hunger', 'hunger']):
        return '2) Zero Hunger'
    elif any(keyword in sdg for keyword in ['good health and well-being', 'healthy', 'health', 'aids']):
        return '3) Good Health and Well-being'
    elif 'quality education' in sdg:
        return '4) Quality Education'
    elif any(keyword in sdg for keyword in ['gender equality', 'women']):
        return '5) Gender Equality'
    elif any(keyword in sdg for keyword in ['clean water and sanitation', 'chemicals', 'water and sanitation']):
        return '6) Clean Water and Sanitation'
    elif any(keyword in sdg for keyword in ['affordable and clean energy', 'energy']):
        return '7) Affordable and Clean Energy'
    elif any(keyword in sdg for keyword in ['decent work and economic growth', 'economic growth']):
        return '8) Decent Work and Economic Growth'
    elif any(keyword in sdg for keyword in ['industry', 'innovation', 'infrastructure']):
        return '9) Industry, Innovation and Infrastructure'
    elif 'reduced inequalities' in sdg or 'inequality' in sdg or 'reduce inequalities' in sdg:
        return '10) Reduced Inequalities'
    elif any(keyword in sdg for keyword in ['sustainable cities', 'cities']):
        return '11) Sustainable Cities and Communities'
    elif any(keyword in sdg for keyword in ['responsible consumption', 'consumption']):
        return '12) Responsible Consumption and Production'
    elif any(keyword in sdg for keyword in ['climate action', 'climate change']):
        return '13) Climate Action'
    elif any(keyword in sdg for keyword in ['life below water', 'oceans', 'life under water']):
        return '14) Life Below Water'
    elif any(keyword in sdg for keyword in ['life on land', 'forest', 'land and soil']):
        return '15) Life on Land'
    elif any(keyword in sdg for keyword in ['peace', 'justice', 'strong institutions','rule of law']):
        return '16) Peace, Justice and Strong Institutions'
    elif any(keyword in sdg for keyword in ['partnerships', 'partnership','sustainable development goals']):
        return '17) Partnerships for the Goals'
    elif any(keyword in sdg for keyword in ['no sdg', 'nan', 'n/a', 'not applicable']):
        return "No SDG"
    elif any(keyword in Link for keyword in ['Synergy']):
        return 'Synergy'
    elif any(keyword in sdg for keyword in ['Trade-off']):
        return 'Trade-off'
    else:
        return sdg


## Pre-prompt

In [ ]:
definitions_separated = [str('SDG1) NO POVERTY: Globally, the number of people living in extreme poverty declined from 36 per cent in 1990 to 10 per cent in 2015. But the pace of change is decelerating and the COVID-19 crisis risks reversing decades of progress in the fight against poverty. New research published by the UNU World Institute for Development Economics Research warns that the economic fallout from the global pandemic could increase global poverty by as much as half a billion people, or 8% of the total human population. This would be the first time that poverty has increased globally in thirty years, since 1990. More than 700 million people, or 10 per cent of the world population, still live in extreme poverty today, struggling to fulfil the most basic needs like health, education, and access to water and sanitation, to name a few. The majority of people living on less than $1.90 a day live in sub-Saharan Africa. Worldwide, the poverty rate in rural areas is 17.2 per cent—more than three times higher than in urban areas. For those who work, having a job does not guarantee a decent living. In fact, 8 per cent of employed workers and their families worldwide lived in extreme poverty in 2018. One out of five children live in extreme poverty. Ensuring social protection for all children and other vulnerable groups is critical to reduce poverty. TARGETS include: 1.1 By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day. 1.2 By 2030, reduce at least by half the proportion of men, women and children of all ages living in poverty in all its dimensions according to national definitions. 1.3 Implement nationally appropriate social protection systems and measures for all, including floors, and by 2030 achieve substantial coverage of the poor and the vulnerable. 1.4 By 2030, ensure that all men and women, in particular the poor and the vulnerable, have equal rights to economic resources, as well as access to basic services, ownership and control over land and other forms of property, inheritance, natural resources, appropriate new technology and financial services, including microfinance. 1.5 By 2030, build the resilience of the poor and those in vulnerable situations and reduce their exposure and vulnerability to climate-related extreme events and other economic, social and environmental shocks and disasters. 1.A Ensure significant mobilization of resources from a variety of sources, including through enhanced development cooperation, in order to provide adequate and predictable means for developing countries, in particular least developed countries, to implement programmes and policies to end poverty in all its dimensions. 1.B Create sound policy frameworks at the national, regional and international levels, based on pro-poor and gender-sensitive development strategies, to support accelerated investment in poverty eradication actions.'),
                         str('SDG2) ZERO HUNGER: After decades of steady decline, the number of people who suffer from hunger – as measured by the prevalence of undernourishment – began to slowly increase again in 2015. Today, more than 820 million people regularly go to bed hungry, of whom about 135 million suffer from acute hunger largely due to man-made conflicts, climate change and economic downturns. The COVID-19 pandemic could now double that number, putting an additional 130 million people at risk of suffering acute hunger by the end of 2020, according to the World Food Programme. With more than a quarter of a billion people potentially at the brink of starvation, swift action needs to be taken to provide food and humanitarian relief to the most at-risk regions. At the same time, a profound change of the global food and agriculture system is needed if we are to nourish the more than 820 million people who are hungry and the additional 2 billion people the world will have by 2050. Increasing agricultural productivity and sustainable food production are crucial to help alleviate the perils of hunger. TARGETS include: 2.1 By 2030, end hunger and ensure access by all people, in particular the poor and people in vulnerable situations, including infants, to safe, nutritious and sufficient food all year round. 2.2 By 2030, end all forms of malnutrition, including achieving, by 2025, the internationally agreed targets on stunting and wasting in children under 5 years of age, and address the nutritional needs of adolescent girls, pregnant and lactating women and older persons. 2.3 By 2030, double the agricultural productivity and incomes of small-scale food producers, in particular women, indigenous peoples, family farmers, pastoralists and fishers, including through secure and equal access to land, other productive resources and inputs, knowledge, financial services, markets and opportunities for value addition and non-farm employment. 2.4 By 2030, ensure sustainable food production systems and implement resilient agricultural practices that increase productivity and production, that help maintain ecosystems, that strengthen capacity for adaptation to climate change, extreme weather, drought, flooding and other disasters and that progressively improve land and soil quality. 2.5 By 2020, maintain the genetic diversity of seeds, cultivated plants and farmed and domesticated animals and their related wild species, including through soundly managed and diversified seed and plant banks at the national, regional and international levels, and promote access to and fair and equitable sharing of benefits arising from the utilization of genetic resources and associated traditional knowledge, as internationally agreed. 2.A Increase investment, including through enhanced international cooperation, in rural infrastructure, agricultural research and extension services, technology development and plant and livestock gene banks in order to enhance agricultural productive capacity in developing countries, in particular least developed countries. 2.B Correct and prevent trade restrictions and distortions in world agricultural markets, including through the parallel elimination of all forms of agricultural export subsidies and all export measures with equivalent effect, in accordance with the mandate of the Doha Development Round. 2.C Adopt measures to ensure the proper functioning of food commodity markets and their derivatives and facilitate timely access to market information, including on food reserves, in order to help limit extreme food price volatility.'),
                         str('SDG3) GOOD HEALTH AND WELL BEING: Ensuring healthy lives and promoting well-being at all ages is essential to sustainable development. Currently, the world is facing a global health crisis unlike any other — COVID-19 is spreading human suffering, destabilizing the global economy and upending the lives of billions of people around the globe. Before the pandemic, major progress was made in improving the health of millions of people. Significant strides were made in increasing life expectancy and reducing some of the common killers associated with child and maternal mortality. But more efforts are needed to fully eradicate a wide range of diseases and address many different persistent and emerging health issues. By focusing on providing more efficient funding of health systems, improved sanitation and hygiene, and increased access to physicians, significant progress can be made in helping to save the lives of millions. Health emergencies such as COVID-19 pose a global risk and have shown the critical need for preparedness. The United Nations Development Programme highlighted huge disparities in countries’ abilities to cope with and recover from the COVID-19 crisis. The pandemic provides a watershed moment for health emergency preparedness and for investment in critical 21st century public services. TARGETS include: 3.1 By 2030, reduce the global maternal mortality ratio to less than 70 per 100,000 live births. 3.2 By 2030, end preventable deaths of newborns and children under 5 years of age, with all countries aiming to reduce neonatal mortality to at least as low as 12 per 1,000 live births and under-5 mortality to at least as low as 25 per 1,000 live births. 3.3 By 2030, end the epidemics of AIDS, tuberculosis, malaria and neglected tropical diseases and combat hepatitis, water-borne diseases and other communicable diseases. 3.4 By 2030, reduce by one third premature mortality from non-communicable diseases through prevention and treatment and promote mental health and well-being. 3.5 Strengthen the prevention and treatment of substance abuse, including narcotic drug abuse and harmful use of alcohol. 3.6 By 2020, halve the number of global deaths and injuries from road traffic accidents. 3.7 By 2030, ensure universal access to sexual and reproductive health-care services, including for family planning, information and education, and the integration of reproductive health into national strategies and programmes. 3.8 Achieve universal health coverage, including financial risk protection, access to quality essential health-care services and access to safe, effective, quality and affordable essential medicines and vaccines for all. 3.9 By 2030, substantially reduce the number of deaths and illnesses from hazardous chemicals and air, water and soil pollution and contamination. 3.A Strengthen the implementation of the World Health Organization Framework Convention on Tobacco Control in all countries, as appropriate. 3.B Support the research and development of vaccines and medicines for the communicable and noncommunicable diseases that primarily affect developing countries, provide access to affordable essential medicines and vaccines, in accordance with the Doha Declaration on the TRIPS Agreement and Public Health, which affirms the right of developing countries to use to the full the provisions in the Agreement on Trade Related Aspects of Intellectual Property Rights regarding flexibilities to protect public health, and, in particular, provide access to medicines for all. 3.C Substantially increase health financing and the recruitment, development, training and retention of the health workforce in developing countries, especially in least developed countries and small island developing States. 3.D Strengthen the capacity of all countries, in particular developing countries, for early warning, risk reduction and management of national and global health risks.'),
                         str('SDG4) QUALITY EDUCATION: Education enables upward socioeconomic mobility and is a key to escaping poverty. Over the past decade, major progress was made towards increasing access to education and school enrollment rates at all levels, particularly for girls. Nevertheless, about 260 million children were still out of school in 2018 — nearly one fifth of the global population in that age group. And more than half of all children and adolescents worldwide are not meeting minimum proficiency standards in reading and mathematics. In 2020, as the COVID-19 pandemic spread across the globe, a majority of countries announced the temporary closure of schools, impacting more than 91 per cent of students worldwide. By April 2020, close to 1.6 billion children and youth were out of school. And nearly 369 million children who rely on school meals needed to look to other sources for daily nutrition. Never before have so many children been out of school at the same time, disrupting learning and upending lives, especially the most vulnerable and marginalised. The global pandemic has far-reaching consequences that may jeopardize hard won gains made in improving global education. TARGETS include: 4.1 By 2030, ensure that all girls and boys complete free, equitable and quality primary and secondary education leading to relevant and Goal-4 effective learning outcomes. 4.2 By 2030, ensure that all girls and boys have access to quality early childhood development, care and preprimary education so that they are ready for primary education. 4.3 By 2030, ensure equal access for all women and men to affordable and quality technical, vocational and tertiary education, including university. 4.4 By 2030, substantially increase the number of youth and adults who have relevant skills, including technical and vocational skills, for employment, decent jobs and entrepreneurship. 4.5 By 2030, eliminate gender disparities in education and ensure equal access to all levels of education and vocational training for the vulnerable, including persons with disabilities, indigenous peoples and children in vulnerable situations. 4.6 By 2030, ensure that all youth and a substantial proportion of adults, both men and women, achieve literacy and numeracy. 4.7 By 2030, ensure that all learners acquire the knowledge and skills needed to promote sustainable development, including, among others, through education for sustainable development and sustainable lifestyles, human rights, gender equality, promotion of a culture of peace and non-violence, global citizenship and appreciation of cultural diversity and of culture’s contribution to sustainable development. 4.A Build and upgrade education facilities that are child, disability and gender sensitive and provide safe, nonviolent, inclusive and effective learning environments for all. 4.B By 2020, substantially expand globally the number of scholarships available to developing countries, in particular least developed countries, small island developing States and African countries, for enrolment in higher education, including vocational training and information and communications technology, technical, engineering and scientific programmes, in developed countries and other developing countries. 4.C By 2030, substantially increase the supply of qualified teachers, including through international cooperation for teacher training in developing countries, especially least developed countries and small island developing states.'),
                         str('SDG5) GENDER EQUALITY: Gender equality is not only a fundamental human right, but a necessary foundation for a peaceful, prosperous and sustainable world. There has been progress over the last decades: More girls are going to school, fewer girls are forced into early marriage, more women are serving in parliament and positions of leadership, and laws are being reformed to advance gender equality. Despite these gains, many challenges remain: discriminatory laws and social norms remain pervasive, women continue to be underrepresented at all levels of political leadership, and 1 in 5 women and girls between the ages of 15 and 49 report experiencing physical or sexual violence by an intimate partner within a 12-month period. The effects of the COVID-19 pandemic could reverse the limited progress that has been made on gender equality and women’s rights.  The coronavirus outbreak exacerbates existing inequalities for women and girls across every sphere – from health and the economy, to security and social protection. Women play a disproportionate role in responding to the virus, including as frontline healthcare workers and carers at home. Women’s unpaid care work has increased significantly as a result of school closures and the increased needs of older people. Women are also harder hit by the economic impacts of COVID-19, as they disproportionately work in insecure labour markets. Nearly 60 per cent of women work in the informal economy, which puts them at greater risk of falling into poverty. The pandemic has also led to a steep increase in violence against women and girls. With lockdown measures in place, many women are trapped at home with their abusers, struggling to access services that are suffering from cuts and restrictions. Emerging data shows that, since the outbreak of the pandemic, violence against women and girls – and particularly domestic violence – has intensified. TARGETS include: 5.1 End all forms of discrimination against all women and girls everywhere. 5.2 Eliminate all forms of violence against all women and girls in the public and private spheres, including trafficking and sexual and other types of exploitation. 5.3 Eliminate all harmful practices, such as child, early and forced marriage and female genital mutilation. 5.4 Recognize and value unpaid care and domestic work through the provision of public services, infrastructure and social protection policies and the promotion of shared responsibility within the household and the family as nationally appropriate. 5.5 Ensure women’s full and effective participation and equal opportunities for leadership at all levels of decisionmaking in political, economic and public life. 5.6 Ensure universal access to sexual and reproductive health and reproductive rights as agreed in accordance with the Programme of Action of the International Conference on Population and Development and the Beijing Platform for Action and the outcome documents of their review conferences. 5.A Undertake reforms to give women equal rights to economic resources, as well as access to ownership and control over land and other forms of property, financial services, inheritance and natural resources, in accordance with national laws. 5.B Enhance the use of enabling technology, in particular information and communications technology, to promote the empowerment of women. 5.C Adopt and strengthen sound policies and enforceable legislation for the promotion of gender equality and the empowerment of all women and girls at all levels.'),
                         str('SDG6) CLEAN WATER AND SANITATION: While substantial progress has been made in increasing access to clean drinking water and sanitation, billions of people—mostly in rural areas—still lack these basic services. Worldwide, one in three people do not have access to safe drinking water, two out of five people do not have a basic hand-washing facility with soap and water, and more than 673 million people still practice open defecation. The COVID-19 pandemic has demonstrated the critical importance of sanitation, hygiene and adequate access to clean water for preventing and containing diseases. Hand hygiene saves lives. According to the World Health Organization, handwashing is one of the most effective actions you can take to reduce the spread of pathogens and prevent infections, including the COVID-19 virus. Yet billions of people still lack safe water sanitation, and funding is inadequate. TARGETS include: 6.1 By 2030, achieve universal and equitable access to safe and affordable drinking water for all. 6.2 By 2030, achieve access to adequate and equitable sanitation and hygiene for all and end open defecation, paying special attention to the needs of women and girls and those in vulnerable situations. 6.3 By 2030, improve water quality by reducing pollution, eliminating dumping and minimizing release of hazardous chemicals and materials, halving the proportion of untreated wastewater and substantially increasing recycling and safe reuse globally. 6.4 By 2030, substantially increase water-use efficiency across all sectors and ensure sustainable withdrawals and supply of freshwater to address water scarcity and substantially reduce the number of people suffering from water scarcity. 6.5 By 2030, implement integrated water resources management at all levels, including through transboundary cooperation as appropriate. 6.6 By 2020, protect and restore water-related ecosystems, including mountains, forests, wetlands, rivers, aquifers and lakes. 6.A By 2030, expand international cooperation and capacity-building support to developing countries in water- and sanitation-related activities and programmes, including water harvesting, desalination, water efficiency, wastewater treatment, recycling and reuse technologies. 6.B Support and strengthen the participation of local communities in improving water and sanitation management.'),
                         str('SDG7) AFFORDABLE AND CLEAN ENERGY: The world is making progress towards Goal 7, with encouraging signs that energy is becoming more sustainable and widely available. Access to electricity in poorer countries has begun to accelerate, energy efficiency continues to improve, and renewable energy is making impressive gains in the electricity sector. Nevertheless, more focused attention is needed to improve access to clean and safe cooking fuels and technologies for 3 billion people, to expand the use of renewable energy beyond the electricity sector, and to increase electrification in sub-Saharan Africa. The Energy Progress Report provides global dashboard to register progress on energy access, energy efficiency and renewable energy. It assesses the progress made by each country on these three pillars and provides a snapshot of how far we are from achieving the 2030 Sustainable Development Goals targets. TARGETS include: 7.1 By 2030, ensure universal access to affordable, reliable and modern energy services. 7.2 By 2030, increase substantially the share of renewable energy in the global energy mix. 7.3 By 2030, double the global rate of improvement in energy efficiency. 7.A By 2030, enhance international cooperation to facilitate access to clean energy research and technology, including renewable energy, energy efficiency and advanced and cleaner fossil-fuel technology, and promote investment in energy infrastructure and clean energy technology. 7.B By 2030, expand infrastructure and upgrade technology for supplying modern and sustainable energy services for all in developing countries, in particular least developed countries, small island developing States, and land-locked developing countries, in accordance with their respective programmes of support.'),
                         str('SDG8) PROMOTE INCLUSIVE AND SUSTAINABLE ECONOMIC GROWTH, EMPLOYMENT AND DECENT WORK FOR ALL: Sustained and inclusive economic growth can drive progress, create decent jobs for all and improve living standards. COVID-19 has disrupted billions of lives and endangered the global economy. The International Monetary Fund (IMF) expects a global recession as bad as or worse than in 2009. As job losses escalate, the International Labor Organization estimates that nearly half of the global workforce is at risk of losing their livelihoods. Even before the outbreak of COVID-19, one in five countries – home to billions of people living in poverty – were likely to see per capita incomes stagnate or decline in 2020. Now, the economic and financial shocks associated with COVID-19—such as disruptions to industrial production, falling commodity prices, financial market volatility, and rising insecurity—are derailing the already tepid economic growth and compounding heightened risks from other factors. TARGETS include: 8.1 Sustain per capita economic growth in accordance with national circumstances and, in particular, at least 7 per cent gross domestic product growth per annum in the least developed countries. 8.2 Achieve higher levels of economic productivity through diversification, technological upgrading and innovation, including through a focus on high-value added and labour-intensive sectors. 8.3 Promote development-oriented policies that support productive activities, decent job creation, entrepreneurship, creativity and innovation, and encourage the formalization and growth of micro-, small- and medium-sized enterprises, including through access to financial services. 8.4 Improve progressively, through 2030, global resource efficiency in consumption and production and endeavour to decouple economic growth from environmental degradation, in accordance with the 10-year framework of programmes on sustainable consumption and production, with developed countries taking the lead. 8.5 By 2030, achieve full and productive employment and decent work for all women and men, including for young people and persons with disabilities, and equal pay for work of equal value. 8.6 By 2020, substantially reduce the proportion of youth not in employment, education or training. 8.7 Take immediate and effective measures to eradicate forced labour, end modern slavery and human trafficking and secure the prohibition and elimination of the worst forms of child labour, including recruitment and use of child soldiers, and by 2025 end child labour in all its forms. 8.8 Protect labour rights and promote safe and secure working environments for all workers, including migrant workers, in particular women migrants, and those in precarious employment. 8.9 By 2030, devise and implement policies to promote sustainable tourism that creates jobs and promotes local culture and products. 8.10 Strengthen the capacity of domestic financial institutions to encourage and expand access to banking, insurance and financial services for all. 8.A Increase Aid for Trade support for developing countries, in particular least developed countries, including through the Enhanced Integrated Framework for Trade-Related Technical Assistance to Least Developed Countries. 8.B By 2020, develop and operationalize a global strategy for youth employment and implement the Global Jobs Pact of the International Labour Organization.'),
                         str('SDG9) BUILD RESILIENT INFRASTRUCTURE, PROMOTE SUSTAINABLE INDUSTRIALIZATION AND FOSTER INNOVATION: Inclusive and sustainable industrialization, together with innovation and infrastructure, can unleash dynamic and competitive economic forces that generate employment and income. They play a key role in introducing and promoting new technologies, facilitating international trade and enabling the efficient use of resources. However, the world still has a long way to go to fully tap this potential. Least developed countries, in particular, need to accelerate the development of their manufacturing sector if they are to meet the 2030 target, and scale up investment in scientific research and innovation. Global manufacturing growth has been steadily declining, even before the outbreak of the COVID-19 pandemic. The pandemic is hitting manufacturing industries hard and causing disruptions in global value chains and the supply of products. Innovation and technological progress are key to finding lasting solutions to both economic and environmental challenges, such as increased resource and energy-efficiency. Globally, investment in research and development (R&D) as a proportion of GDP increased from 1.5 per cent in 2000 to 1.7 per cent in 2015 and remained almost unchanged in 2017, but was only less than 1 per cent  in developing regions. In terms of communications infrastructure, more than half of the world’s population is now online and almost the entire world population lives in an area covered by a mobile network. It is estimated that in 2019, 96.5 per cent were covered by at least a 2G network. The coronavirus pandemic has revealed the urgent need for resilient infrastructure. The Asian Development Bank notes that critical infrastructure in the region remains far from adequate in many countries, despite the rapid economic growth and development the region has experienced over the past decade. The Economic and Social Survey of Asia and the Pacific highlights that making infrastructure resilient to disasters and climate change will require an additional investment of $434 billion per year. This sum may need to be even greater in some subregions, such as the Pacific small island developing states. TARGETS include: 9.1 Develop quality, reliable, sustainable and resilient infrastructure, including regional and transborder infrastructure, to support economic development and human well-being, with a focus on affordable and equitable access for all. 9.2 Promote inclusive and sustainable industrialization and, by 2030, significantly raise industry’s share of employment and gross domestic product, in line with national circumstances, and double its share in least developed countries. 9.3 Increase the access of small-scale industrial and other enterprises, in particular in developing countries, to financial services, including affordable credit, and their integration into value chains and markets. 9.4 By 2030, upgrade infrastructure and retrofit industries to make them sustainable, with increased resource-use efficiency and greater adoption of clean and environmentally sound technologies and industrial processes, with all countries taking action in accordance with their respective capabilities. 9.5 Enhance scientific research, upgrade the technological capabilities of industrial sectors in all countries, in particular developing countries, including, by 2030, encouraging innovation and substantially increasing the number of research and development workers per 1 million people and public and private research and development spending. 9.A Facilitate sustainable and resilient infrastructure development in developing countries through enhanced financial, technological and technical support to African countries, least developed countries, landlocked developing countries and small island developing States 18. 9.B Support domestic technology development, research and innovation in developing countries, including by ensuring a conducive policy environment for, inter alia, industrial diversification and value addition to commodities. 9.C Significantly increase access to information and communications technology and strive to provide universal and affordable access to the Internet in least developed countries by 2020.'),
                         str('SDG10) REDUCE INEQUALITY WITHIN AND AMONG COUNTRIES: Reducing inequalities and ensuring no one is left behind are integral to achieving the Sustainable Development Goals. Inequality within and among countries is a persistent cause for concern. Despite some positive signs toward reducing inequality in some dimensions, such as reducing relative income inequality in some countries and preferential trade status benefiting lower-income countries, inequality still persists. COVID-19 has deepened existing inequalities, hitting the poorest and most vulnerable communities the hardest. It has put a spotlight on economic inequalities and fragile social safety nets that leave vulnerable communities to bear the brunt of the crisis.  At the same time, social, political and economic inequalities have amplified the impacts of the pandemic. On the economic front, the COVID-19 pandemic has significantly increased global unemployment and dramatically slashed workers’ incomes. COVID-19 also puts at risk the limited progress that has been made on gender equality and women’s rights over the past decades. Across every sphere, from health to the economy, security to social protection, the impacts of COVID-19 are exacerbated for women and girls simply by virtue of their sex. Inequalities are also deepening for vulnerable populations in countries with weaker health systems and those facing existing humanitarian crises. Refugees and migrants, as well as indigenous peoples, older persons, people with disabilities and children are particularly at risk of being left behind. And hate speech targeting vulnerable groups is rising. TARGETS include: 10.1 By 2030, progressively achieve and sustain income growth of the bottom 40 per cent of the population at a rate higher than the national average. 10.2 By 2030, empower and promote the social, economic and political inclusion of all, irrespective of age, sex, disability, race, ethnicity, origin, religion or economic or other status. 10.3 Ensure equal opportunity and reduce inequalities of outcome, including by eliminating discriminatory laws, policies and practices and promoting appropriate legislation, policies and action in this regard. 10.4 Adopt policies, especially fiscal, wage and social protection policies, and progressively achieve greater equality. 10.5 Improve the regulation and monitoring of global financial markets and institutions and strengthen the implementation of such regulations. 10.6 Ensure enhanced representation and voice for developing countries in decision-making in global international economic and financial institutions in order to deliver more effective, credible, accountable and legitimate institutions. 10.7 Facilitate orderly, safe, regular and responsible migration and mobility of people, including through the implementation of planned and well-managed migration policies. 10.A Implement the principle of special and differential treatment for developing countries, in particular least developed countries, in accordance with World Trade Organization agreements. 10.B Encourage official development assistance and financial flows, including foreign direct investment, to States where the need is greatest, in particular least developed countries, African countries, small island developing States and landlocked developing countries, in accordance with their national plans and programmes. 10.C By 2030, reduce to less than 3 per cent the transaction costs of migrant remittances and eliminate remittance corridors with costs higher than 5 per cent.'),
                         str('SDG11) MAKE CITIES INCLUSIVE, SAFE, RESILIENT AND SUSTAINABLE: The world is becoming increasingly urbanized. Since 2007, more than half the world’s population has been living in cities, and that share is projected to rise to 60 per cent by 2030. Cities and metropolitan areas are powerhouses of economic growth—contributing about 60 per cent of global GDP. However, they also account for about 70 per cent of global carbon emissions and over 60 per cent of resource use. Rapid urbanization is resulting in a growing number of slum dwellers, inadequate and overburdened infrastructure and services (such as waste collection and water and sanitation systems, roads and transport), worsening air pollution and unplanned urban sprawl. The impact of COVID-19 will be most devastating in poor and densely populated urban areas, especially for the one billion people living in informal settlements and slums worldwide, where overcrowding also makes it difficult to follow recommended measures such as social distancing and self-isolation. The UN food agency, FAO, warned that hunger and fatalities could rise significantly in urban areas, without measures to ensure that poor and vulnerable residents have access to food. TARGETS include: 11.1 By 2030, ensure access for all to adequate, safe and affordable housing and basic services and upgrade slums. 11.2 By 2030, provide access to safe, affordable, accessible and sustainable transport systems for all, improving road safety, notably by expanding public transport, with special attention to the needs of those in vulnerable situations, women, children, persons with disabilities and older persons. 11.3 By 2030, enhance inclusive and sustainable urbanization and capacity for participatory, integrated and sustainable human settlement planning and management in all countries. 11.4 Strengthen efforts to protect and safeguard the world’s cultural and natural heritage. 11.5 By 2030, significantly reduce the number of deaths and the number of people affected and substantially decrease the direct economic losses relative to global gross domestic product caused by disasters, including water-related disasters, with a focus on protecting the poor and people in vulnerable situations. 11.6 By 2030, reduce the adverse per capita environmental impact of cities, including by paying special attention to air quality and municipal and other waste management. 11.7 By 2030, provide universal access to safe, inclusive and accessible, green and public spaces, in particular for women and children, older persons and persons with disabilities. 11.A Support positive economic, social and environmental links between urban, peri-urban and rural areas by strengthening national and regional development planning. 11.B By 2020, substantially increase the number of cities and human settlements adopting and implementing integrated policies and plans towards inclusion, resource efficiency, mitigation and adaptation to climate change, resilience to disasters, and develop and implement, in line with the Sendai Framework for Disaster Risk Reduction 2015-2030, holistic disaster risk management at all levels. 11.C Support least developed countries, including through financial and technical assistance, in building sustainable and resilient buildings utilizing local materials.'),
                         str('SDG12) ENSURE SUSTAINABLE CONSUMPTION AND PRODUCTION PATTERNS: Worldwide consumption and production — a driving force of the global economy — rest on the use of the natural environment and resources in a way that continues to have destructive impacts on the planet. Economic and social progress over the last century has been accompanied by environmental degradation that is endangering the very systems on which our future development — indeed, our very survival — depends. A few facts and figures: Each year, an estimated one third of all food produced – equivalent to 1.3 billion tonnes worth around $1 trillion – ends up rotting in the bins of consumers and retailers, or spoiling due to poor transportation and harvesting practices. If people worldwide switched to energy efficient light bulbs the world would save US$120 billion annually. Should the global population reach 9.6 billion by 2050, the equivalent of almost three planets could be required to provide the natural resources needed to sustain current lifestyles. TARGETS include: 12.1 Implement the 10-year framework of programmes on sustainable consumption and production, all countries taking action, with developed countries taking the lead, taking into account the development and capabilities of developing countries. 12.2 By 2030, achieve the sustainable management and efficient use of natural resources. 12.3 By 2030, halve per capita global food waste at the retail and consumer levels and reduce food losses along production and supply chains, including post-harvest losses. 12.4 By 2020, achieve the environmentally sound management of chemicals and all wastes throughout their life cycle, in accordance with agreed international frameworks, and significantly reduce their release to air, water and soil in order to minimize their adverse impacts on human health and the environment. 12.5 By 2030, substantially reduce waste generation through prevention, reduction, recycling and reuse. 12.6 Encourage companies, especially large and transnational companies, to adopt sustainable practices and to integrate sustainability information into their reporting cycle. 12.7 Promote public procurement practices that are sustainable, in accordance with national policies and priorities. 12.8 By 2030, ensure that people everywhere have the relevant information and awareness for sustainable development and lifestyles in harmony with nature. 12.A Support developing countries to strengthen their scientific and technological capacity to move towards more sustainable patterns of consumption and production. 12.B Develop and implement tools to monitor sustainable development impacts for sustainable tourism that creates jobs and promotes local culture and products. 12.C Rationalize inefficient fossil-fuel subsidies that encourage wasteful consumption by removing market distortions, in accordance with national circumstances, including by restructuring taxation and phasing out those harmful subsidies, where they exist, to reflect their environmental impacts, taking fully into account the specific needs and conditions of developing countries and minimizing the possible adverse impacts on their development in a manner that protects the poor and the affected communities.'),
                         str('SDG13) TAKE URGENT ACTION TO COMBAT CLIMATE CHANGE AND ITS IMPACTS: 2019 was the second warmest year on record and the end of the warmest decade (2010- 2019) ever recorded. Carbon dioxide (CO2) levels and other greenhouse gases in the atmosphere rose to new records in 2019. Climate change is affecting every country on every continent. It is disrupting national economies and affecting lives. Weather patterns are changing, sea levels are rising, and weather events are becoming more extreme. Although greenhouse gas emissions are projected to drop about 6 per cent in 2020 due to travel bans and economic slowdowns resulting from the COVID-19 pandemic, this improvement is only temporary. Climate change is not on pause. Once the global economy begins to recover from the pandemic, emissions are expected to return to higher levels. Saving lives and livelihoods requires urgent action to address both the pandemic and the climate emergency. The Paris Agreement, adopted in 2015, aims to strengthen the global response to the threat of climate change by keeping a global temperature rise this century well below 2 degrees Celsius above pre-industrial levels. The agreement also aims to strengthen the ability of countries to deal with the impacts of climate change, through appropriate financial flows, a new technology framework and an enhanced capacity building framework. TARGETS include: 13.1 Strengthen resilience and adaptive capacity to climate-related hazards and natural disasters in all countries. 13.2 Integrate climate change measures into national policies, strategies and planning. 13.3 Improve education, awareness-raising and human and institutional capacity on climate change mitigation, adaptation, impact reduction and early warning. 13.A Implement the commitment undertaken by developed-country parties to the United Nations Framework Convention on Climate Change to a goal of mobilizing jointly $100 billion annually by 2020 from all sources to address the needs of developing countries in the context of meaningful mitigation actions and transparency on implementation and fully operationalize the Green Climate Fund through its capitalization as soon as possible. 13.B Promote mechanisms for raising capacity for effective climate change-related planning and management in least developed countries and small island developing States, including focusing on women, youth and local and marginalized communities.'),
                         str('SDG14) CONSERVE AND SUSTAINABLY USE THE OCEANS, SEAS AND MARINE RESOURCES: The ocean drives global systems that make the Earth habitable for humankind. Our rainwater, drinking water, weather, climate, coastlines, much of our food, and even the oxygen in the air we breathe, are all ultimately provided and regulated by the sea. Careful management of this essential global resource is a key feature of a sustainable future. However, at the current time, there is a continuous deterioration of coastal waters owing to pollution, and ocean acidification is having an adversarial effect on the functioning of ecosystems and biodiversity. This is also negatively impacting small scale fisheries. Saving our ocean must remain a priority. Marine biodiversity is critical to the health of people and our planet. Marine protected areas need to be effectively managed and well-resourced and regulations need to be put in place to reduce overfishing, marine pollution and ocean acidification. TARGETS include: 14.1 By 2025, prevent and significantly reduce marine pollution of all kinds, in particular from land-based activities, including marine debris and nutrient pollution. 14.2 By 2020, sustainably manage and protect marine and coastal ecosystems to avoid significant adverse impacts, including by strengthening their resilience, and take action for their restoration in order to achieve healthy and productive oceans. 14.3 Minimize and address the impacts of ocean acidification, including through enhanced scientific cooperation at all levels. 14.4 By 2020, effectively regulate harvesting and end overfishing, illegal, unreported and unregulated fishing and destructive fishing practices and implement science-based management plans, in order to restore fish stocks in the shortest time feasible, at least to levels that can produce maximum sustainable yield as determined by their biological characteristics. 14.5 By 2020, conserve at least 10 per cent of coastal and marine areas, consistent with national and international law and based on the best available scientific information. 14.6 By 2020, prohibit certain forms of fisheries subsidies which contribute to overcapacity and overfishing, eliminate subsidies that contribute to illegal, unreported and unregulated fishing and refrain from introducing new such subsidies, recognizing that appropriate and effective special and differential treatment for developing and least developed countries should be an integral part of the World Trade Organization fisheries subsidies negotiation. 14.7 By 2030, increase the economic benefits to Small Island developing States and least developed countries from the sustainable use of marine resources, including through sustainable management of fisheries, aquaculture and tourism. 14.A Increase scientific knowledge, develop research capacity and transfer marine technology, taking into account the Intergovernmental Oceanographic Commission Criteria and Guidelines on the Transfer of Marine Technology, in order to improve ocean health and to enhance the contribution of marine biodiversity to the development of developing countries, in particular small island developing States and least developed countries. 14.B Provide access for small-scale artisanal fishers to marine resources and markets. 14.C Enhance the conservation and sustainable use of oceans and their resources by implementing international law as reflected in UNCLOS, which provides the legal framework for the conservation and sustainable use of oceans and their resources, as recalled in paragraph 158 of The Future We Want.'),
                         str('SDG15) SUSTAINABLY MANAGE FORESTS, COMBAT DESERTIFICATION, HALT AND REVERSE LAND DEGRADATION, HALT BIODIVERSITY LOSS: Nature is critical to our survival: nature provides us with our oxygen, regulates our weather patterns, pollinates our crops, produces our food, feed and fibre. But it is under increasing stress. Human activity has altered almost 75 per cent of the earth’s surface, squeezing wildlife and nature into an ever-smaller corner of the planet. Around 1 million animal and plant species are threatened with extinction – many within decades – according to the 2019 Global Assessment Report on Biodiversity and Ecosystem Service. The report called for transformative changes to restore and protect nature. It found that the health of ecosystems on which we and all other species depend is deteriorating more rapidly than ever, affecting  the very foundations of our economies, livelihoods, food security, health and quality of life worldwide. Deforestation and desertification – caused by human activities and climate change – pose major challenges to sustainable development and have affected the lives and livelihoods of millions of people. Forests are vitally important for sustaining life on Earth, and play a major role in the fight against climate change. And investing in land restoration is critical for improving livelihoods, reducing vulnerabilities, and reducing risks for the economy. The health of our planet also plays an important role in the emergence of zoonotic diseases, i.e. diseases that are transmissible between animals and humans. As we continue to encroach on fragile ecosystems, we bring humans into ever-greater contact with wildlife, enabling pathogens in wildlife to spill over to livestock and humans, increasing the risk of disease emergence and amplification. TARGETS include: 15.1 By 2020, ensure the conservation, restoration and sustainable use of terrestrial and inland freshwater ecosystems and their services, in particular forests, wetlands, mountains and drylands, in line with obligations under international agreements. 15.2 By 2020, promote the implementation of sustainable management of all types of forests, halt deforestation, restore degraded forests and substantially increase afforestation and reforestation globally. 15.3 By 2030, combat desertification, restore degraded land and soil, including land affected by desertification, drought and floods, and strive to achieve a land degradation-neutral world. 15.4 By 2030, ensure the conservation of mountain ecosystems, including their biodiversity, in order to enhance their capacity to provide benefits that are essential for sustainable development. 15.5 Take urgent and significant action to reduce the degradation of natural habitats, halt the loss of biodiversity and, by 2020, protect and prevent the extinction of threatened species. 15.6 Promote fair and equitable sharing of the benefits arising from the utilization of genetic resources and promote appropriate access to such resources, as internationally agreed. 15.7 Take urgent action to end poaching and trafficking of protected species of flora and fauna and address both demand and supply of illegal wildlife products. 15.8 By 2020, introduce measures to prevent the introduction and significantly reduce the impact of invasive alien species on land and water ecosystems and control or eradicate the priority species. 15.9 By 2020, integrate ecosystem and biodiversity values into national and local planning, development processes, poverty reduction strategies and accounts. 15.A Mobilize and significantly increase financial resources from all sources to conserve and sustainably use biodiversity and ecosystems. 15.B Mobilize significant resources from all sources and at all levels to finance sustainable forest management and provide adequate incentives to developing countries to advance such management, including for conservation and reforestation. 15.C Enhance global support for efforts to combat poaching and trafficking of protected species, including by increasing the capacity of local communities to pursue sustainable livelihood opportunities.'),
                         str('SDG16) PEACE, JUSTICE AND STRONG INSTITUTIONS: Conflict, insecurity, weak institutions and limited access to justice remain a great threat to sustainable development. The number of people fleeing war, persecution and conflict exceeded 70 million in 2018, the highest level recorded by the UN refugee agency (UNHCR) in almost 70 years. In 2019, the United Nations tracked 357 killings and 30 enforced disappearances of human rights defenders, journalists and trade unionists in 47 countries. And the births of around one in four children under age 5 worldwide are never officially recorded, depriving them of a proof of legal identity crucial for the protection of their rights and for access to justice and social services. TARGETS include: 16.1 Significantly reduce all forms of violence and related death rates everywhere. 16.2 End abuse, exploitation, trafficking and all forms of violence against and torture of children. 16.3 Promote the rule of law at the national and international levels and ensure equal access to justice for all. 16.4 By 2030, significantly reduce illicit financial and arms flows, strengthen the recovery and return of stolen assets and combat all forms of organized crime. 16.5 Substantially reduce corruption and bribery in all their forms. 16.6 Develop effective, accountable and transparent institutions at all levels. 16.7 Ensure responsive, inclusive, participatory and representative decision-making at all levels. 16.8 Broaden and strengthen the participation of developing countries in the institutions of global governance. 16.9 By 2030, provide legal identity for all, including birth registration. 16.10 Ensure public access to information and protect fundamental freedoms, in accordance with national legislation and international agreements. 16.A Strengthen relevant national institutions, including through international cooperation, for building capacity at all levels, in particular in developing countries, to prevent violence and combat terrorism and crime. 16.B Promote and enforce non-discriminatory laws and policies for sustainable development.'),
                         str('SDG17) PARTNERSHIPS FOR THE GOALS: The SDGs can only be realized with strong global partnerships and cooperation. A successful development agenda requires inclusive partnerships — at the global, regional, national and local levels — built upon principles and values, and upon a shared vision and shared goals placing people and the planet at the centre. Many countries require Official Development Assistance to encourage growth and trade. Yet, aid levels are falling and donor countries have not lived up to their pledge to ramp up development finance. Due to the COVID-19 pandemic, the global economy is projected to contract sharply, by 3 per cent, in 2020, experiencing its worst recession since the Great Depression. Strong international cooperation is needed now more than ever to ensure that countries have the means to recover from the pandemic, build back better and achieve the Sustainable Development Goals. TARGETS include: Finance 17.1 Strengthen domestic resource mobilization, including through international support to developing countries, to improve domestic capacity for tax and other revenue collection.17.2 Developed countries to implement fully their official development assistance commitments, including the commitment by many developed countries to achieve the target of 0.7 per cent of ODA/GNI to developing countries and 0.15 to 0.20 per cent of ODA/GNI to least developed countries ODA providers are encouraged to consider setting a target to provide at least 0.20 per cent of ODA/GNI to least developed countries. 17.3 Mobilize additional financial resources for developing countries from multiple sources. 17.4 Assist developing countries in attaining long-term debt sustainability through coordinated policies aimed at fostering debt financing, debt relief and debt restructuring, as appropriate, and address the external debt of highly indebted poor countries to reduce debt distress. 17.5 Adopt and implement investment promotion regimes for least developed countries. Technology 17.6 Enhance North-South, South-South and triangular regional and international cooperation on and access to science, technology and innovation and enhance knowledge sharing on mutually agreed terms, including through improved coordination among existing mechanisms, in particular at the United Nations level, and through a global technology facilitation mechanism. 17.7 Promote the development, transfer, dissemination and diffusion of environmentally sound technologies to developing countries on favourable terms, including on concessional and preferential terms, as mutually agreed. 17.8 Fully operationalize the technology bank and science, technology and innovation capacity-building mechanism for least developed countries by 2017 and enhance the use of enabling technology, in particular information and communications technology. Capacity building 17.9 Enhance international support for implementing effective and targeted capacity-building in developing countries to support national plans to implement all the sustainable development goals, including through North-South, South-South and triangular cooperation. Trade 17.10 Promote a universal, rules-based, open, non-discriminatory and equitable multilateral trading system under the World Trade Organization, including through the conclusion of negotiations under its Doha Development Agenda. 17.11 Significantly increase the exports of developing countries, in particular with a view to doubling the least developed countries’ share of global exports by 2020. 17.12 Realize timely implementation of duty-free and quota-free market access on a lasting basis for all least developed countries, consistent with World Trade Organization decisions, including by ensuring that preferential rules of origin applicable to imports from least developed countries are transparent and simple, and contribute to facilitating market access. Systemic issues Policy and institutional coherence 17.13 Enhance global macroeconomic stability, including through policy coordination and policy coherence. 17.14 Enhance policy coherence for sustainable development. 17.15 Respect each country’s policy space and leadership to establish and implement policies for poverty eradication and sustainable development. Multi-stakeholder partnerships 17.16 Enhance the global partnership for sustainable development, complemented by multi-stakeholder partnerships that mobilize and share knowledge, expertise, technology and financial resources, to support the achievement of the sustainable development goals in all countries, in particular developing countries. 17.17 Encourage and promote effective public, public-private and civil society partnerships, building on the experience and resourcing strategies of partnerships. Data, monitoring and accountability 17.18 By 2020, enhance capacity-building support to developing countries, including for least developed countries and small island developing States, to increase significantly the availability of high-quality, timely and reliable data disaggregated by income, gender, age, race, ethnicity, migratory status, disability, geographic location and other characteristics relevant in national contexts. 17.19 By 2030, build on existing initiatives to develop measurements of progress on sustainable development that complement gross domestic product, and support statistical capacity-building in developing countries.')]

## Loop through Excel

In [ ]:
# Define the number of samples
start_index = 0
end_index = 81244

# Loop through the indexes of the DataFrame
for index in range(start_index, end_index):
    # Each text from the excel file
    print(f"Processing index: {index}")

    paragraph = df.loc[index, 'FullText']
    file_name = df.loc[index, 'File Name']
    file_index = df.loc[index, 'Unnamed: 0']

    sdg_a = df.loc[index, 'SDG A']
    sdg_b = df.loc[index, 'SDG B']

    # Ensure that SDG 6 is the one causing the trade-off (in SDG B)
    if sdg_b == '6) Clean Water and Sanitation':
        relevant_sdgs = [sdg_a]  # Analyze how SDG 6 affects SDG A
    else:
        print(f"Skipping index {index} - SDG B is not 6.")
        continue

    if len(relevant_sdgs) == 0:
        print(f"Skipping index {index} - No relevant SDGs found.")
        continue

    # Where to save the new response
    destination_path = os.path.join(interaction_water_folder, f"{index}.txt")
    if os.path.exists(destination_path):
        print(f"Skipping {destination_path} because it already exists...")
        continue

    # Updated prompt with the three categories
    prompt = f"""

    You are provided with the content of a scientific article on water. This article describes **synergistic relationships** between **SDG 6: Clean Water and Sanitation** and **other Sustainable Development Goals (SDGs).**

    **Article Title:** {file_name}

    **Your task:**
    Identify and classify the **type of synergy** where achieving SDG 6 benefits the attainment of another SDG. Specifically, determine whether the synergy falls into one of the following categories:

    ---

    ### **Categories of Synergy:**

    1️. **Genuine Synergy**
      This refers to cases where **achieving SDG 6 has a direct and well-supported positive impact on another SDG.** The relationship is clear, logical, and backed by strong evidence.
      **Examples include:**
      - Improved water access reducing disease rates (**SDG 3: Health**).
      - Sustainable water use enhancing agricultural productivity (**SDG 2: Food Security**).
      - Clean water access improving educational outcomes (**SDG 4: Quality Education**).


    2️. **Indirect or Context-Dependent Synergy**
      This refers to cases where **the synergy exists but depends on external factors, assumptions, or is not fully demonstrated in the article.** The connection might be logical, but the evidence is weak or indirect.
      **Examples include:**
      - Clean water **potentially** leading to economic growth (**SDG 8**), but without clear mechanisms.
      - Sanitation improvements **possibly** fostering stronger institutions (**SDG 16**), but the link is not well detailed.
      - Water conservation measures **having an impact** on energy sustainability (**SDG 7**), but requiring further policy interventions.

    3️. **Weak Synergy (Weak or Dubious Synergy)**
      This refers to cases where **the relationship is vague, misinterpreted, or not actually a synergy.** The supposed link might be:
      - **Too indirect** (the effect is too far removed from SDG 6).
      - **Causality is reversed** (the other SDG supports SDG 6, not the other way around).
      - **Poorly justified** (the article does not provide enough evidence).

      **Examples include:**
      - Stating that water policies depend on strong institutions (**SDG 16**), but not demonstrating how SDG 6 actually strengthens governance.
      - Claiming that industrial water use boosts innovation (**SDG 9**), but lacking clear evidence.
      - Suggesting that water management inherently promotes peace (**SDG 16**), without proving a direct effect.


    ###  **Guidelines for your analysis:**
    1️. **If the synergy is clearly stated and supported by evidence,** classify it as **Genuine Synergy.**
    2️. **If the synergy exists but requires assumptions or context-specific conditions,** classify it as **Indirect or Context-Dependent Synergy.**
    3️. **If the link is weak, misinterpreted, or not a real synergy,** classify it as **Weak Synergy.**

    ---

    **Original article:**

    Article: "{paragraph}"

    ---

    **Output Format:**
    """
    for sdg in relevant_sdgs:
        prompt += f"""
          - **SDG Pair:** SDG 6 (Clean Water and Sanitation) - SDG {sdg}
            - **Article Title:** {file_name}
            - **Article Index:** {file_index}
            - **Type of Synergy:** [Genuine Synergy / Indirect or Context-Dependent Synergy / Weak Synergy]
            - **Explanation:** Provide an exact quote from the article (if available) and a detailed analysis explaining why this is a synergy.
            - **Inferred Evidence (if applicable):** If the synergy is not directly stated, explain any **indirect evidence** or **logical reasoning** that suggests a connection.
        """

    # Verify the generated prompt
    #print(f"Generated prompt for index {index}:\n", prompt)

    try:
        # Generate the response with Gemini
        response = model.generate_content(prompt, safety_settings=safety_settings, generation_config=generation_config)
        print(response.text)

        # Save the response in a text file
        with open(destination_path, 'w') as file:
            file.write(response.text)

    except Exception as e:
        if '500' in str(e):
            print("=========================================== Exception 500 =========================================== ")
        elif '429' in str(e):  # If it occurs more than 20 times consecutively, change the key or wait until the next day
            print("=========================================== Exception 429 =========================================== ")
            time.sleep(5)
        elif 'Read timed out' in str(e):  # Restart session and run everything
            print("=========================================== Exception 600 =========================================== ")
        else:
            print(f"Skipping due to error: {e}")


In [ ]:
# Parameters
block_size = 10000  # number of files per block

# Path where the generated .txt files are located
interaction_water_folder = "/content/drive/My Drive/TrabajoWater/Raw_Results_Flash_002/Prompt0_synergy2_prueba1"
output_folder = "/content/drive/My Drive/TrabajoWater"

# Load the original file to obtain the metadata
path_excel = os.path.join(output_folder, "synergy-parte2.xlsx")
df = pd.read_excel(path_excel)

# Get all the files
txt_files = sorted([f for f in os.listdir(interaction_water_folder) if f.endswith(".txt")])
total_files = len(txt_files)

# Split into blocks
num_blocks = (total_files + block_size - 1) // block_size

for block_num in range(num_blocks):
    start_idx = block_num * block_size
    end_idx = min((block_num + 1) * block_size, total_files)
    block_files = txt_files[start_idx:end_idx]

    output_path = os.path.join(output_folder, f"synergyreves_results_block_{block_num+1}.xlsx")
    if os.path.exists(output_path):
        print(f" Block {block_num+1} already exists. Skipping...")
        continue

    results = []
    print(f"\n Processing block {block_num+1}/{num_blocks} - Files {start_idx+1} to {end_idx}")

    for file in tqdm(block_files, desc=f"Block {block_num+1}"):
        try:
            index = int(file.replace(".txt", ""))
        except ValueError:
            print(f" Unexpected file name: {file}")
            continue

        file_path = os.path.join(interaction_water_folder, file)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
        except Exception as e:
            print(f" Error reading {file}: {e}")
            continue

        # Extract metadata from the original Excel
        try:
            article_title = df.loc[index, 'File Name']
            sdg_a = df.loc[index, 'SDG A']
            sdg_b = df.loc[index, 'SDG B']
        except:
            article_title = ""
            sdg_a = ""
            sdg_b = ""

        # Split by sections if there are multiple SDG pairs
        entries = content.split("- **SDG Pair:**")
        for entry in entries[1:]:  # Skip the first (empty before the first pair)

            # Initialize fields
            synergy_type = ""
            explanation = ""
            inferred_evidence = ""
            sdg_pair = ""

            lines = entry.strip().split("\n")
            sdg_pair_line = lines[0].strip()
            if sdg_pair_line.startswith("SDG"):
                sdg_pair = sdg_pair_line

            for i, line in enumerate(lines):
                if "Type of Synergy:" in line:
                    synergy_type = line.split("Type of Synergy:")[-1].strip()
                elif "Explanation:" in line:
                    explanation = line.split("Explanation:")[-1].strip()
                elif "Inferred Evidence" in line:
                    inferred_evidence = line.split("Inferred Evidence:")[-1].strip()

            results.append({
                "Index": index,
                "Article Title": article_title,
                "SDG A": sdg_a,
                "SDG B": sdg_b,
                "SDG Pair": sdg_pair,
                "Type of Synergy": synergy_type,
                "Explanation": explanation,
                "Inferred Evidence": inferred_evidence
            })

    # Save to Excel
    df_block = pd.DataFrame(results)
    df_block.to_excel(output_path, index=False)
    print(f" Saved: {output_path}")

In [ ]:
# Path where the block excels are stored
output_folder = "/content/drive/My Drive/TrabajoWater"

# Search for all block files
excel_files = sorted([f for f in os.listdir(output_folder) if f.startswith("synergyreves_results_block_") and f.endswith(".xlsx")])

# Read and concatenate all excels
all_dataframes = []

for file in excel_files:
    file_path = os.path.join(output_folder, file)
    df = pd.read_excel(file_path)
    all_dataframes.append(df)

# Combine all DataFrames
df_combined = pd.concat(all_dataframes, ignore_index=True)

# Save to a single Excel file
final_output_path = os.path.join(output_folder, "synergy_results_Part2.xlsx")
df_combined.to_excel(final_output_path, index=False)

print(f" Combined file saved to: {final_output_path}")